In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130510  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,1.451275e+06
2018-02-28,1.163838e+06
2018-03-31,1.112772e+06
2018-04-30,1.245269e+06
2018-05-31,1.295940e+06
2018-06-30,1.235673e+06
2018-07-31,1.314077e+06
2018-08-31,1.504072e+06
2018-09-30,1.213019e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    1.451275e+06
 2018-02-28    1.163838e+06
 2018-03-31    1.112772e+06
 2018-04-30    1.245269e+06
 2018-05-31    1.295940e+06
 2018-06-30    1.235673e+06
 2018-07-31    1.314077e+06
 2018-08-31    1.504072e+06
 2018-09-30    1.213019e+06
 2018-10-31    1.227929e+06
 2018-11-30    1.112022e+06
 2018-12-31    1.301594e+06
 2019-01-31    1.412389e+06
 2019-02-28    1.131750e+06
 2019-03-31    1.141340e+06
 2019-04-30    1.054230e+06
 2019-05-31    1.204527e+06
 2019-06-30    1.195791e+06
 2019-07-31    1.242237e+06
 2019-08-31    1.333901e+06
 2019-09-30    1.030932e+06
 2019-10-31    1.139834e+06
 2019-11-30    1.099737e+06
 2019-12-31    1.120760e+06
 2020-01-31    1.376933e+06
 2020-02-29    9.519741e+05
 2020-03-31    1.074473e+06
 2020-04-30    8.927965e+05
 2020-05-31    1.088274e+06
 2020-06-30    1.054378e+06
 2020-07-31    1.198135e+06
 2020-08-31    1.301750e+06
 2020-09-30    1.083000e+06
 2020-10-31    1.309605e+06
 2020-11-30    1.274585e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -1.249759
p-value : 0.651833
Critical Values :
	1%: -3.620918
	5%: -2.943539
	10%: -2.610400


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.544585
p-value : 0.000002
Critical Values :
	1%: -3.620918
	5%: -2.943539
	10%: -2.610400


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1115.149, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1115.465, Time=0.11 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1115.685, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1117.969, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1119.857, Time=0.11 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1119.692, Time=0.12 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1114.825, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1116.069, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1117.705, Time=0.05 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1119.505, Time=0.13 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1121.412, Time=0.24 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1115.569, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1117.485, Time=0.13 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1119.517, Time=0.26 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1120.577, Time=0.31 sec
 ARIMA(3,0

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(1,0,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -554.386
Method:                       css-mle   S.D. of innovations         130614.288
Date:                Sat, 11 Sep 2021   AIC                           1114.772
Time:                        02:30:19   BIC                           1119.985
Sample:                             0   HQIC                          1116.683
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.215e+06   2.65e+04     45.905      0.000    1.16e+06    1.27e+06
ar.L1.y        0.2435      0.156      1.565      0.118      -0.061       0.548
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([1247748.65226848, 1223215.51775458]), array([130614.28756822, 134431.1226019 ]), array([[ 991749.35276841, 1503747.95176855],
       [ 959735.35905357, 1486695.67645559]]))
